# Imports

In [1]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint

# Initial Setup

To begin I've chosen to focus on getting all of the stats from one game.

Due to personal preference I've chosen to look at the preseason game between the Dallas Fuel and the Houston Outlaws. The page on winstons lab for the game can be found here:

[Preseason: Fuel vs. Outlaws](https://www.winstonslab.com/matches/match.php?id=2367)

The individual match id for the site is 2367. We'll try to keep everything as modular as possible during prototyping.

In [2]:
BASE_MATCH_URL = "https://www.winstonslab.com/matches/match.php"

In [3]:
match_id = "2367"

In [4]:
full_url = BASE_MATCH_URL + "?id=" + match_id

## Get the base html using requests

In [5]:
response = requests.get(full_url)
html = BeautifulSoup(response.text, "lxml")

# Overview stats

In [6]:
div = html.findAll("div", {"class": "side-by-side-stats"})[0] # table containing detailed stats
detail = div.findAll("div", recursive=False)[2]

In [7]:
detail.text

''

Ok so this is empty because It loads the data after clicked detailed stats. I guess we'll have to use Selenium and PhantomJS

In [8]:
from selenium import webdriver
import time

In [9]:
driver = webdriver.Chrome("/Applications/chromedriver")

JK using Chrome. I need to find a chrome headless driver.

In [10]:
driver.get(full_url)
time.sleep(5)

In [11]:
detail_button = driver.find_element_by_class_name("showDetailedStatsBut")

In [12]:
detail_button.text

'Detailed Stats'

In [14]:
detail_button.click()

In [15]:
detail_stats = driver.find_element_by_id("detailedStatsDiv_0")

In [16]:
detail_stats.text

'Color Code: Top 25% Bottom 25%\nDetailed Stats\nPlayer\nHero\nTime\nRating\nFWin%\nPTK\nK/10\nD/10\nU/10\nTTCU\nKPU\nUE\nUOOF\nFK\nFD\nImpact\nBani Zenyatta 00:08:14 1451 55.56% 19.15% 10.93 6.07 4.86 91 0 44.44 25% 22% 11% 1355\nBoink Mercy 00:44:32 1206 55.32% 0.56% 0.22 3.82 3.59 87 0 21.6 12.5% 0% 9% 1116\nTaimou Roadhog 00:30:25 1193 57.14% 25.77% 8.22 4.6 4.27 74 0 5.36 8.33% 21% 11% 1098\nRawkus Zenyatta 00:49:16 1181 62.75% 11.34% 4.47 5.07 4.87 92 0 6 5% 4% 4% 772\nJAKE Junkrat 00:28:21 1137 54.17% 38.37% 11.64 6.7 4.59 77 0 12.5 0% 33% 21% 1451\nEffect Zarya 00:17:46 1124 40% 19.51% 4.5 2.81 3.94 72 0 -20 0% 0% 10% 773\nMuma Orisa 00:27:25 1121 60.87% 9.57% 3.28 3.65 3.28 98 0 16.91 0% 9% 9% 1048\nJAKE Soldier: 76 00:08:16 1110 70% 20% 9.68 3.63 4.84 96 0 30 0% 0% 0% 823\nEffect Tracer 00:47:26 1098 46.55% 29.83% 11.38 7.17 8.43 57 0 5.83 22.22% 12% 10% 964\nBani Mercy 00:13:15 1066 78.57% 0% 0 3.02 3.77 99 0 21.43 0% 0% 0% 422\nHarryHook Zenyatta 00:24:30 1065 37.04% 10.42%

In [17]:
page_source = driver.page_source

In [18]:
new_html = BeautifulSoup(page_source, "lxml")

In [21]:
div = new_html.findAll("div", {"class": "side-by-side-stats"})[0] # table containing detailed stats
detail = div.findAll("div", recursive=False)[2]

In [23]:
stat_table = detail.table

In [24]:
table_headers = stat_table.thead
table_data = stat_table.tbody

In [25]:
header_elems = table_headers.findAll("th")

In [26]:
header_list = []
for header in header_elems:
    header_list.append(header.text)
pprint(header_list)

['Player',
 'Hero',
 'Time',
 'Rating',
 'FWin%',
 'PTK',
 'K/10',
 'D/10',
 'U/10',
 'TTCU',
 'KPU',
 'UE',
 'UOOF',
 'FK',
 'FD',
 'Impact',
 'Elim/10',
 'dmg/10',
 'heal/10',
 'ObjTime/10',
 'ObjKills/10']


In [27]:
row_list = table_data.findAll("tr")

In [28]:
for row in row_list[0:3]:
    elems = row.findAll("td")
    print("-----------------------")
    for i, item in enumerate(elems[0:]):
        print("{}: {}".format(header_list[i], item.text.strip()))

-----------------------
Player: Bani
Hero: Zenyatta
Time: 00:08:14
Rating: 1451
FWin%: 55.56%
PTK: 19.15%
K/10: 10.93
D/10: 6.07
U/10: 4.86
TTCU: 91
KPU: 0
UE: 44.44
UOOF: 25%
FK: 22%
FD: 11%
Impact: 1355
Elim/10: N/A
dmg/10: N/A
heal/10: N/A
ObjTime/10: N/A
ObjKills/10: N/A
-----------------------
Player: Boink
Hero: Mercy
Time: 00:44:32
Rating: 1206
FWin%: 55.32%
PTK: 0.56%
K/10: 0.22
D/10: 3.82
U/10: 3.59
TTCU: 87
KPU: 0
UE: 21.6
UOOF: 12.5%
FK: 0%
FD: 9%
Impact: 1116
Elim/10: N/A
dmg/10: N/A
heal/10: N/A
ObjTime/10: N/A
ObjKills/10: N/A
-----------------------
Player: Taimou
Hero: Roadhog
Time: 00:30:25
Rating: 1193
FWin%: 57.14%
PTK: 25.77%
K/10: 8.22
D/10: 4.6
U/10: 4.27
TTCU: 74
KPU: 0
UE: 5.36
UOOF: 8.33%
FK: 21%
FD: 11%
Impact: 1098
Elim/10: N/A
dmg/10: N/A
heal/10: N/A
ObjTime/10: N/A
ObjKills/10: N/A
